In [1]:
import numpy as np
import pandas as pd
import cython
import math as m
from scipy.stats import norm, gmean
%load_ext Cython

In [2]:
def Geometric_Asian_Option_price(S0,sigma,K,r,T,Nt,flag):
    ###########################################################################
    #This is a closed form solution for geometric Asian options
    #
    #S0 = Current price of underlying asset
    #sigma = Volatility
    #K = Strike price
    #r = Risk-free rate
    #T = Time to maturity
    #Nt = Time intervals
    ###########################################################################
    adj_sigma=sigma*m.sqrt((2*Nt+1)/(6*(Nt+1)))
    rho=0.5*(r-(sigma**2)*0.5+adj_sigma**2)
    d1 = (m.log(S0/K)+(rho+0.5*adj_sigma**2)*T)/(adj_sigma*m.sqrt(T))
    d2 = (m.log(S0/K)+(rho-0.5*adj_sigma**2)*T)/(adj_sigma*m.sqrt(T))
    if (flag=="Call"):
        price = m.exp(-r*T)*(S0*m.exp(rho*T)*norm.cdf(d1)-K*norm.cdf(d2)) # Important to note that BSM has CDF
    elif (flag =="Put"):
        price = m.exp(-r*T)*(K*norm.cdf(-d2)-S0*m.exp(rho*T)*norm.cdf(-d1))
    return(price)

In [3]:
S0=100
sigma=0.2
K=100
r=0.03
T=5
Nt=T*252 #number of trading days until maturity

Geometric_Asian_Option_price(S0,sigma,K,r,T,Nt,flag="Put")

6.7505109558164493

In [4]:
Geometric_Asian_Option_price(S0,sigma,K,r,T,Nt,flag="Call")

11.919430915155033

In [5]:
%%cython
import time
start_time = time.time()
cdef extern from "math.h":
    double sqrt(double x)
    double exp(double x)
import numpy as np
cimport numpy as np
# cimport math as m
cdef double S0
cdef double r
cdef double sigma
cdef int steps
cdef int T
np.random.seed(1)
cpdef sim_stocks(S0, r,sigma,steps =252,T=5):
    cdef int nSims = 10000
    cdef int sim_steps = steps*T
    cdef double dt = 1/steps
    cdef int div = 0
    cdef double mu = (r-div-0.5*sigma*sigma)*dt
    sigma = sigma*sqrt(dt)
    St = np.zeros(shape=(sim_steps,nSims))
    St[0,] = S0
    for i in range(1,sim_steps):
            for j in range(0,nSims):
                e = np.random.randn(1)
                St[i,j] = St[i-1,j]*exp(mu+sigma*e)
    return(St)

sim_stocks = sim_stocks(100,0.03,sigma=0.3,steps = 252,T=5)
print("The cythonized execution takes {0} seconds ".format(time.time() - start_time))

The cythonized execution takes 125.68697333335876 seconds 


In [6]:
# In our analysis we are only focusing on geometric call option
sim_stocks = pd.DataFrame(sim_stocks)
prices = []
X = []
steps=252
sim_steps = steps*5
for i in range(0,sim_steps):
    prices.append(gmean(sim_stocks [i])) # Taking the geometric mean
    X.append(max(prices[i]-K,0)*m.exp(-r*T))
geometric_opt_price  = np.mean(X)
geometric_opt_price

15.305407759417266

In [7]:
prices = []
Y = []
steps=252
sim_steps = steps*5
for i in range(0,sim_steps):
    prices.append(np.mean(sim_stocks [i])) # Taking the arithmetic mean
    Y.append(max(prices[i]-K,0)*m.exp(-r*T))
arithmetic_opt_price  = np.mean(Y)
arithmetic_opt_price

17.726172152386773

In [8]:
numerator = []
denominator = []
for i in range(1,sim_steps):
    numerator.append((Y[i]-arithmetic_opt_price)* (X[i]-geometric_opt_price))
    denominator.append((X[i]-geometric_opt_price)**2)

B = sum(numerator)/sum(denominator)
B

1.1503647088468885

In [9]:
Pgsim = geometric_opt_price
Pg = Geometric_Asian_Option_price(S0,sigma,K,r,T,Nt,flag="Call")
Error = Pgsim - Pg 
Error

3.3859768442622329

In [10]:
Pasim = arithmetic_opt_price
Pa = Pasim - B*Error
# The modified arithmetic option price
Pa

13.831063885774743

In [11]:
Pasim-Pa

3.8951082666120307